# Predicting Location via Indoor Positioning System 
## Analysis and Modeling in Python *prototypes and drabbles
This script and subsequent notebooks are the Python conversion of the Indoor Location case study from the book, 'Data Science in R' by Nolan and Lang. http://rdatasciencecases.org/code.html <BR>
* Team: Jonathan Tan, Lance Dacy, Reannan Mcdaniel, Shawn Jung 
* Last Update: 5/14/2020 

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from __future__ import print_function
#from statsmodels.distributions.empirical_distribution import ECDF
%matplotlib inline 

In [3]:
# Let's load dataframes
#offline_pivot = pd.read_csv('./data/offline_pivot.csv')
#online_pivot = pd.read_csv('./data/online_pivot.csv')
offline_pivot = pd.read_csv('D:/Code/QTW Case Study 1/offline_pivot.csv')
online_pivot = pd.read_csv('D:/Code/QTW Case Study 1/online_pivot.csv')

# re-add pos_x and pos_y for distance error calculation in later stage
offline_pivot[['pos_x', 'pos_y']] = offline_pivot['pos_xy'].str.split('-',n=2, expand=True).astype(float)
online_pivot[['pos_x', 'pos_y']] = online_pivot['pos_xy'].str.split('-',n=2, expand=True).astype(float)

In [4]:
test0 = online_pivot.iloc[0,1:8]
train0 = offline_pivot.iloc[0,1:8]

In [6]:
#offline_pivot.head()

In [7]:
offline_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pos_xy             166 non-null    object 
 1   00:0f:a3:39:dd:cd  166 non-null    float64
 2   00:0f:a3:39:e1:c0  166 non-null    float64
 3   00:14:bf:3b:c7:c6  166 non-null    float64
 4   00:14:bf:b1:97:81  166 non-null    float64
 5   00:14:bf:b1:97:8a  166 non-null    float64
 6   00:14:bf:b1:97:8d  166 non-null    float64
 7   00:14:bf:b1:97:90  166 non-null    float64
 8   pos_x              166 non-null    float64
 9   pos_y              166 non-null    float64
dtypes: float64(9), object(1)
memory usage: 12.4+ KB


In [9]:
#RF test
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

#single class prediction on pos_x, then pos_y??
y = offline_pivot.iloc[:, 8] 
x = offline_pivot.iloc[:, 1:8]

cv = StratifiedKFold(n_splits = 10)
#significant of n_estimators? number of trees?
clf = RandomForestClassifier(n_estimators= 100, random_state = 1)
acc = cross_val_score(clf, x, y = y, cv = cv)

print("average Accuracy For pos_X= ", acc.mean()*100, "+/-", acc.std()*100)
#confusion matrix for mean accuracy?

y = offline_pivot.iloc[:, 9] #for pos_y
x = offline_pivot.iloc[:, 1:8]

cv = StratifiedKFold(n_splits = 10)

clf = RandomForestClassifier(n_estimators= 100, random_state = 1)
acc = cross_val_score(clf, x, y = y, cv = cv)

print("average Accuracy For pos_Y= ", acc.mean()*100, "+/-", acc.std()*100)
#hmm not great, 22% acc for x, 44% acc for y for Random Forest Classifier with 100 "branches" and 10 splits

average Accuracy For pos_X=  22.316176470588232 +/- 7.885236659728786
average Accuracy For pos_Y=  44.338235294117645 +/- 16.601371519611863


In [11]:
#single class prediction on pos_x, then pos_y?? 
#   #   #   #
#doesn't work
#   #   #   #
#y = offline_pivot.iloc[:, 0].to_frame() #use combined position column as predictor
#x = offline_pivot.iloc[:, 1:8]

#cv = StratifiedKFold(n_splits = 2)
#significant of n_estimators? number of trees?
#clf = RandomForestClassifier(n_estimators= 100, random_state = 1)
#acc = cross_val_score(clf, x, y = y, cv = cv)

#print("average Accuracy", acc.mean()*100, "+/-", acc.std()*100)

In [13]:
#m3mbers in each class
#from collections import Counter
#Counter(offline_pivot.iloc[:, 0]).keys()
#Counter(offline_pivot.iloc[:, 0]).values()
#so using the combined (x, y) coords looks like they are all unique? not suitable for clustering

In [14]:
#how to visualize results from this format?

In [15]:
#knn split class prediction 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

y = offline_pivot.iloc[:, 8] #just on pos_x 
x = offline_pivot.iloc[:, 1:8]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 83812)

knn = NearestNeighbors(n_neighbors = 2)
knn.fit(x_train, y_train)

A = knn.kneighbors_graph(x)
A.toarray()
#need to fit some kind of metric to this

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
#kmeans
from sklearn.cluster import KMeans
import numpy as np

x2 = offline_pivot.iloc[:, 1:8] #fit kmeans to predictors, 7 macs
y2 =offline_pivot.iloc[:, 9:10]

cls = KMeans(n_clusters = 7, init = 'k-means++', random_state = 1)
cls.fit(x2)
newfeature = cls.labels_

X = offline_pivot.iloc[:, 1:8]
X = np.column_stack((X,pd.get_dummies(newfeature))) #use for comparison with acc cross val

acc = cross_val_score(clf, X, y = y2, cv = cv)
print('KMeans Pos_X accuracy  = {:.2f}%'.format(acc[0]*100))
print('KMeans Pos_Y accuracy  = {:.2f}%'.format(acc[1]*100))
#not better than random forest

KMeans Pos_X accuracy  = 42.17%
KMeans Pos_Y accuracy  = 31.33%


In [18]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [144]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

y = offline_pivot.iloc[:, 8:10] 
x = offline_pivot.iloc[:, 1:8]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 83812)

classifier = BinaryRelevance(GaussianNB()) #binary relevance multiclassifier 
#knn = KNeighborsClassifier(n_neighbors = 4)
#classifier = MultiOutputClassifier(knn, n_jobs = -1)
classifier.fit(x_train, y_train) #fit
predictions = classifier.predict(x_test) #predict

#both to array
y_test = y_test.values #convert dataframe to array #so predictions from knn come out as array, but accuracy score doesn't support multiclass?
predictions = predictions.toarray()

#try converting both to sparse? 
#y_test_sparse = sparse.csc_matrix(y_test) #convert y_test t0 sparse? csc
#predictions_sparse = sparse.csc_matrix(predictions) 

#try matrix conversion?
#y_test = np.matrix(y_test)
#predictions = np.matrix(predictions)

from sklearn.metrics import accuracy_score
#accuracy_score(y_test, predictions)
#accuracy_score(y_test_sparse, predictions_sparse) ok both sparse to accuracy_score gives a len() error?

print('Hamming Score:', hamming_score(y_test, predictions)) #Hamming score is the difference in positions?
#print('Subset Accuracy:', sklearn.metrics.accuracy_score(y_test, predictions))
#print('Hamming Loss:', sklearn.metrics.hamming_loss(y_test,predictions))


Hamming Score: 0.8823529411764706


Hamming Distance Example

karolin
kathrin
  |||      Different at 3 positions so a distance of 3

1011101
1001001
  | |      Distance of 2

2173896
2233796
 || |      Distance of 3


how to reconcile accuracy with hamming score? how to get accuracy out of multiclass/multilabel output?

In [20]:
#multiclass multivariable output RF
from sklearn.datasets import make_multilabel_classification

y = offline_pivot.iloc[:, 8:10] 
x = offline_pivot.iloc[:, 1:8]

cv = StratifiedKFold(n_splits = 10)

clf = RandomForestClassifier(n_estimators= 100, random_state = 1)
#acc = cross_val_score(clf, x, y = y, cv = cv)
#multiclass, multioutput not compatible with cross val score
#print("average Accuracy = ", acc.mean()*100, "+/-", acc.std()*100)

In [19]:
def distance(test_array, train_df, number_of_returns=1):
    dist = []
    for i in range(train_df.shape[0]):
        d = np.sqrt(np.sum(np.square((np.array(test_array) - np.array(train_df)[i]))))
        dist.append(d)
    
    dist_result = np.array(dist)
    
    # return top n lowest values

    return dist_result.argsort()[0:number_of_returns][::-1]





In [20]:
test0_pred = distance(test0, offline_pivot.iloc[:,1:8], 1)

print('predicted location:', offline_pivot.iloc[test0_pred]['pos_xy'])
print('original location:', online_pivot.iloc[0]['pos_xy'])

predicted location: 18    1.0-2.0
Name: pos_xy, dtype: object
original location: 0.0-0.05
